# DSCI 511

## Assignment 4 _(120 points)_
## E-reader enhancement with Wikification and Project Gutenberg
For this assignment your work will setup a complete pipeline for the backend of an e-reader enhancement. Specifically, you'll use an API service known as Wikifier to embed hyperlink references to encyclopedia articles within the content areas of html open-domain eBooks.

### A. _(40 points)_ Getting the relevant API's in order

In [83]:
#Libraries in use:
import pandas as pd
import requests
import urllib.parse, urllib.request
import csv 
from bs4 import BeautifulSoup
from pprint import pprint
import re
import csv, json
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from IPython.core.display import HTML
import emoji 
import os
from IPython.display import IFrame

__A1.__ _(5 points)_ Throughout this problem, we're going to be working with the Wikifier API, which is "a web service that takes a text document as input and annotates it with links to relevant Wikipedia concepts". So, before we can begin with the actual work, it's necessary to request access to this API at the following link: http://wikifier.org/info.html. Read this documentation and familiarize yourself with it, and make sure to register for a `USER_KEY`.
Store your `USER_KEY` in the below cell:

In [2]:
USER_KEY = 'xnmoymphovtdrmqxkvazxarlzhxzwr'

__A2.__ _(10 points)_ Now, we'd like to incorporate some Project Gutenberg data. You may recall that each book in the Project Gutenberg collection has a unique ID associated with it. We'd like to write a function that takes as input the ID of a specific Project Gutenberg book and then downloads and consequently stores a full HTML version (with images) of the book. \[__Hint__: To sort out the url structure find and observe the URLs of a few (HTML-format) books on Gutenberg: https://www.gutenberg.org/ and generalize the pattern in your code, bringing in the book's ID number as a an argument into the final URL as needed.\]

In [3]:
def GutenbergDLer(GutenbergID):
    BookURL = "http://www.gutenberg.org/files/"+GutenbergID+"/"+GutenbergID+"-h/"+GutenbergID+"-h.htm"
    html_Book = requests.get(BookURL)
    return html_Book

__A3.__ _(5 points)_ Let's use the function you wrote in the previous part to download a book! Let's grab _Alice in Wonderland_ (ID number `19033`):

In [4]:
html_Alice = GutenbergDLer("19033")

In [5]:
def save_html_file(GutenbergID):
    htmlBook = GutenbergDLer(GutenbergID)
    Html_file= open("BookID"+GutenbergID+".htm","w")
    Html_file.write(htmlBook.text)
    Html_file.close()

In [6]:
save_html_file("19033")

__A4.__ _(15 points)_ In addition to accessing books, we're going to have to make requests to Wikifier, ultimately to the paragraphs of _Alice in Wonderland_. Create a function that takes a block of text as input, and outputs the JSON response given by the Wikification API (per the following specifications) interpreted as a Python object (deserialized). __Important__: For full credit you must read the Wikifier docs: http://wikifier.org/info.html; specifically, you must determine how to build a URL that sets the `'applyPageRankSqThreshold'` field to `True`, the corresponding `'pageRankSqThreshold'` field to `0.8`, and the language field to `'en'`.

In [7]:
def Wikify(text):
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", "en"),
        ("userKey", USER_KEY),
        ("pageRankSqThreshold", "%g" % 0.8), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    return response    

In [8]:
DrexelWikify = Wikify(
    "Drexel University is a private research university with its main campus"+
    " located in the University City neighborhood of Philadelphia, Pennsylvania"+
    ", United States. It was founded in 1891 by Anthony J. Drexel, a noted financier"+
    " and philanthropist. Founded as Drexel Institute of Art, Science, and Industry")

In [9]:
type(DrexelWikify)

dict

In [10]:
DrexelWikify.keys()

dict_keys(['annotations', 'spaces', 'words', 'normWords', 'ranges', 'minPageRank', 'maxDfThreshold', 'altLabelSetsUsed', 'linkSourcesAvailable', 'timeTotalMs', 'timeAnnotationMs', 'timeBuildJsonMs', 'incIdxTimestampUtc', 'fnRegIdx', 'fnWikiData', 'settings', 'language'])

In [11]:
print(DrexelWikify["words"])

['Drexel', 'University', 'is', 'a', 'private', 'research', 'university', 'with', 'its', 'main', 'campus', 'located', 'in', 'the', 'University', 'City', 'neighborhood', 'of', 'Philadelphia', ',', 'Pennsylvania', ',', 'United', 'States', '.', 'It', 'was', 'founded', 'in', '1891', 'by', 'Anthony', 'J', '.', 'Drexel', ',', 'a', 'noted', 'financier', 'and', 'philanthropist', '.', 'Founded', 'as', 'Drexel', 'Institute', 'of', 'Art', ',', 'Science', ',', 'and', 'Industry']


In [12]:
print(DrexelWikify["spaces"])

['', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '', ' ', '', ' ', ' ', '', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '', ' ', '', ' ', ' ', ' ', ' ', ' ', '', ' ', ' ', ' ', ' ', ' ', ' ', '', ' ', '', ' ', ' ', '']


In [13]:
pprint(DrexelWikify["annotations"])

[{'dbPediaIri': 'http://dbpedia.org/resource/Drexel_University',
  'dbPediaTypes': ['Agent',
                   'Place',
                   'Organisation',
                   'University',
                   'EducationalInstitution'],
  'lang': 'en',
  'pageRank': 0.04405086906225279,
  'secLang': 'en',
  'secTitle': 'Drexel University',
  'secUrl': 'http://en.wikipedia.org/wiki/Drexel_University',
  'support': [{'chFrom': 0,
               'chTo': 16,
               'entropy': 0.2412578517720322,
               'pMentionGivenSurface': 0.675933280381255,
               'pageRank': 0.01713757418971146,
               'prbConfidence': 0.7074427084630097,
               'wFrom': 0,
               'wTo': 1},
              {'chFrom': 256,
               'chTo': 271,
               'entropy': 0,
               'pMentionGivenSurface': 0.3877551020408163,
               'pageRank': 0.009831120942758526,
               'prbConfidence': 1,
               'wFrom': 44,
               'wTo': 45},
 

In [14]:
for annotation in Wikify(
    "Drexel University is a private research university with its main campus"+
    " located in the University City neighborhood of Philadelphia, Pennsylvania"+
    ", United States. It was founded in 1891 by Anthony J. Drexel, a noted financier"+
    " and philanthropist. Founded as Drexel Institute of Art, Science, and Industry")["annotations"]:
    print("%s (%s)" % (annotation["title"], annotation["url"]))

Drexel University (http://en.wikipedia.org/wiki/Drexel_University)
Research university (http://en.wikipedia.org/wiki/Research_university)
Philadelphia (http://en.wikipedia.org/wiki/Philadelphia)
Pennsylvania (http://en.wikipedia.org/wiki/Pennsylvania)


__A5.__ _(5 points)_ Check to make sure your function works! Load the small block of text provided under `./data/example_text.txt`, and apply your `Wikify()` function. Print out the result. Does everything look correct, according to the docs? Discuss the output in the response box.

<font color=blue>The output of the function is a dictionary that has annotation key. We can print the Wiki pages by looping over all the annotations and we can extract the URL for each page. One of the links is not correct; the phrase "end of" is linked to a wiki page called Moonsault!</font>

In [15]:
example_text = open("./data/example_text.txt", "r").read()

In [16]:
pprint(example_text)

("Hey smilin' strange\n"
 "You're lookin' happily deranged\n"
 'Could you settle to shoot me?\n'
 'Or have you picked your target yet?\n'
 'Hey Sandy\n'
 "Don't you talk back,\n"
 'Hey Sandy\n'
 'Four feet away\n'
 "End of speech, it's the end of the day\n"
 "We was only funnin'\n"
 "But guiltily I thought you had it comin'\n")


In [17]:
WikifiedExample = Wikify(example_text)

In [18]:
pprint(WikifiedExample)

{'altLabelSetsUsed': [],
 'annotations': [{'dbPediaIri': 'http://dbpedia.org/resource/Hey_Sandy',
                  'dbPediaTypes': ['Person', 'Agent', 'Artist'],
                  'lang': 'en',
                  'pageRank': 0.1080767492300702,
                  'secLang': 'en',
                  'secTitle': 'Hey Sandy',
                  'secUrl': 'http://en.wikipedia.org/wiki/Hey_Sandy',
                  'support': [{'chFrom': 118,
                               'chTo': 126,
                               'entropy': 0,
                               'pMentionGivenSurface': 0.3846153846153846,
                               'pageRank': 0.04410080617528433,
                               'prbConfidence': 1,
                               'wFrom': 24,
                               'wTo': 25},
                              {'chFrom': 149,
                               'chTo': 157,
                               'entropy': 0,
                               'pMentionGivenSurface': 0.384

In [19]:
for annotation in WikifiedExample["annotations"]:
    print("%s (%s)" % (annotation["title"], annotation["url"]))

Hey Sandy (http://en.wikipedia.org/wiki/Hey_Sandy)
Moonsault (http://en.wikipedia.org/wiki/Moonsault)


### B. _(30 points)_ Link embedding

__B1.__ _(5 points)_ To work with the Wikifier output we'll need to go through a few steps. Since the annotationd (embedded links) provided by Wikifer reference specific document indices, i.e., for words (of their tokenization) and characters. We'll work on the character level because it's more precise. To get this off of the ground, make a function called `build_doc(wiki)`, that takes a Wikifier output (`wiki`) and builds up the original document from the `wiki['words']` and `wiki['spaces']` fields. \[__Hint.__ These two fields alternate,  with `wiki['spaces']` having exactly one more element than `wiki['words']`, i.e., a rebuilt document will join the elements of the two with the first element coming from `wiki['spaces']`\].

In [20]:
def build_doc(wiki):
    Original_Text = ""
    for i in range(len(wiki['spaces'])):
        Original_Text = Original_Text + wiki['spaces'][i]
        if i<= len(wiki["words"])-1:
            Original_Text = Original_Text + wiki["words"][i]
        else:
            continue
    return Original_Text

In [21]:
pprint(build_doc(WikifiedExample))

("Hey smilin' strange\n"
 "You're lookin' happily deranged\n"
 'Could you settle to shoot me?\n'
 'Or have you picked your target yet?\n'
 'Hey Sandy\n'
 "Don't you talk back,\n"
 'Hey Sandy\n'
 'Four feet away\n'
 "End of speech, it's the end of the day\n"
 "We was only funnin'\n"
 "But guiltily I thought you had it comin'\n")


__B2.__ _(5 points)_ Now that the document is back together we want to collect the links from the annotations. Write a function called `get_links(wiki)` that processes the `annotation` objects from the `wiki['annotations']` field (a list of `annotation` objects), and creates/outputs a data object called links, which is a list of tuples:
```
links = [(chFrom, chTo, hyperlink), ...]
```
Hyperlinks will come from `annotation["url"]`, and the `chFrom` and `chTo` elements represent character ranges corresponding to embeddings of `annotation["url"]` in the document. Note that tthere are multiple character ranges: one for each `support` object in `annotation["support"]` (also a list).  Specifically, each `support` object in the annotation field has two character indices keyed as `support["chFrom"]` and `support["chTo"]` which designate a range (in character indices) over which the hyperlink might well be embedded. 

In [22]:
def get_links(wiki):
    links = []
    for annotation in wiki["annotations"]:
        for eachsupport in annotation["support"]:
            links.append((eachsupport["chFrom"], eachsupport["chTo"], annotation["url"]))
    return links

In [23]:
get_links(WikifiedExample)

[(118, 126, 'http://en.wikipedia.org/wiki/Hey_Sandy'),
 (149, 157, 'http://en.wikipedia.org/wiki/Hey_Sandy'),
 (174, 179, 'http://en.wikipedia.org/wiki/Moonsault')]

__B3.__ _(5 points)_ Your next job is to create an `embed_link(doc, link)` function that takes the full document output from __B1__, and, a given link (list element) from __B2__ outputs an updated document with hyperlink embedded. Exhibit the function of `embed_link` using `IPython`'s `HTML()` function: `from IPython.core.display import HTML`

In [24]:
def hyperlinker(doc, startchar_index, endchar_index, url):
    link = doc[startchar_index:endchar_index+1]
    hyperlink = "<a href="+url+">"+link+"</a>"
    return hyperlink

In [25]:
def embed_link(doc, link):
    start = 0
    doc1 = ""
    for eachlink in link:
        doc1 = (doc1 + 
                doc[start:eachlink[0]] +
                hyperlinker(doc, eachlink[0], eachlink[1], eachlink[2]))
        start = eachlink[1]+1
    doc2 = doc1+doc[start:]
    return HTML(doc2)

In [26]:
embed_link(build_doc(WikifiedExample), get_links(WikifiedExample))

__B4.__ _(5 points)_ The responses received from Wikifier are really only _predictions_ of where links should go. This means that multiple link predictions might span overlapping portions of source text. So, we'll have to make sure our code does some light decision making to avoid overlapping hyperlinks and broken html code. Review the Wikifier response data and discuss any viable approach to dealing with the possible overlapping link problem in the response box below. \[__Hint__: One such approach is taken in problem __B5__ that you may discuss here. If you do so, you are required to provide a detailed description of _how_ __B5__ resolves the overlapping-link problem and any limitations of its approach.\]

__Note__: Even if you have not yet generated any Wikifier output through Problem A, you may complete this part for full credit using the sample data provided in `./data/example_annotations.json`.

<font color=blue>__Approach:__ In order to avoid overlapping links, we could make a list of all the starting and stopping indices of the urls and check if this list is sorted in a stricktly increasing order. If the list is not increasing we may face the overlapping problem. If the list is monotonically increasing, we have two adjacent distinct links.</font><br ><br >
<font color=blue>__Limitations:__ This approach prevents overlapping, but if the end index of one overlapping link is before the end index of the previous link the hyperlinks in the final document is not placed correct. I have shown one instance of this problem by feeding a faulty link list to the test function at the end of this section.</font>

__B5.__ _(10 points)_ Here is where we're going to build a wrapper function, called `embed_links()`, to manage the link embedding process, being sure to avoid the overlapping link problem. Specifically, your function _must_ do the following, in order:

1. Take a wikifier, `wiki`, response as input.
2. Build the document using __B1__ and storing the output as a string called `doc`
3. Apply __B2__'s `get_links()` function, storing the `links` output 
4. _Reverse_ sort the `link`  by their ending points, i.e., `support["chTo"]` values
5. Loop through the _sorted_ `links` 
6. Conditionally use __B4__'s `embed_link()` function to embed links into `doc` in the loop. Specifically, only embed a link if its `support["chTo"]` value is less than the previous one's `support["chFrom"]` value, or the link to embed is the first in the loop.
7. `return` the link-enhanced doc and exhibit it's output using `HTML()`.

__Note__: If you have not yet completed __Problem A__, you can complete this part for full credit using the sample file, `./data/example_annotations.json`. This file has the correct input format of data for your function (a Wikifier json output).

In [27]:
def embed_links(wiki):
    doc = build_doc(wiki)
    links = get_links(wiki)
    sorted_links = sorted(links, key=lambda tup: tup[1], reverse = True)
    valid_links = []
    for i in range(len(sorted_links)-1):
        if sorted_links[i+1][1]<sorted_links[i][0]:
            valid_links.append(sorted_links[i])
    valid_links.append(sorted_links[-1])        
    new_links = valid_links[::-1]
    return embed_link(doc, new_links)

In [28]:
embed_links(WikifiedExample)

In [29]:
type(embed_links(WikifiedExample))

IPython.core.display.HTML

<font color=red>Here, I want to make a temporary list of faulty links(Since I do not have a text sample with this problem, I am feeding a faulty link list to a test function similar to the one that we are using):</font>

In [30]:
faultyLinklist = [(118, 126, 'http://en.wikipedia.org/wiki/Hey_Sandy'),
 (149, 157, 'http://en.wikipedia.org/wiki/Hey_Sandy'),
 (150, 155, 'http://en.wikipedia.org/wiki/Moonsault')]

In [31]:
def test_embed_links(wiki):
    doc = build_doc(wiki)
    links = faultyLinklist
    sorted_links = sorted(links, key=lambda tup: tup[1], reverse = True)
    valid_links = []
    for i in range(len(sorted_links)-1):
        if sorted_links[i+1][1]<sorted_links[i][0]:
            valid_links.append(sorted_links[i])
    valid_links.append(sorted_links[-1])        
    new_links = valid_links[::-1]
    return embed_link(doc, new_links)

In [32]:
test_embed_links(WikifiedExample)

**It works 😎 but it has a bug 🐞**

__BONUS__ _(5 points)_ The `embed_links()` function's design overcomes an additional challenge. Your job here is to explain in detail how it does so. Here's the challenge: since we are using character indices to insert the links over ranges of the input document, adding the markup required for html _necessarily_ creates a longer document and throws off the original document's character indices from their original positions. How does __B5__'s design avoid this _index modification_ problem and simply utilize the exact character indices provided by Wikifier?

<font color=blue>This function uses the wiki instead of the html document. Wiki is a dictionary with words and spaces and the embed_links function rebuilds the document using the words and spaces. Therefore, the document that we work on is the raw text without any html markup. The indices extracted are based on the location of the hyperlinks in the pure text without any markup.</font>

__BONUS__ _(15 pts)_ Modify your code from __B5__ to handle the overlapping link (and potentially index modification) problem(s) differently and exhibit its output using the `HTML()` function. You may use any strategy distinct from those used in __B5__, but must _additionally_ (_for 5  of the __BONUS__ points_) describe sufficient details to explain your approach's distinctness, along with _how_ your approach satisfies these problems in the response below. \[__Hint__. There's a useful field in the `support` objects held under the `"pMentionGivenSurface"` key; think thresholds and numerical comparisions, and modify the `get_links()` function to make this information available with each link.\]

<font color=blue>pMentionGivenSurface is the probability that, when a link appears in the Wikipedia with this particular subrange as its anchor text, it points to the Wikipedia page corresponding to the current annotation.
We can extract this value for each link and if the value is more than a threshold, such as 1e-2, the link is probably reliable.<br ><br >
As we expected, the faulty link "moonsault" for the phrase "End of" is removed.</font>

In [127]:
def another_get_links(wiki):
    links = []
    validlinks = []
    for annotation in wiki["annotations"]:
        for eachsupport in annotation["support"]:
            links.append((
                eachsupport["chFrom"], eachsupport["chTo"], annotation["url"], eachsupport['pMentionGivenSurface']))
    for eachlink in links:
        if eachlink[3]>=1e-2:
            validlinks.append(eachlink)
        else:
            pass
    return validlinks

In [128]:
another_get_links(WikifiedExample)

[(118, 126, 'http://en.wikipedia.org/wiki/Hey_Sandy', 0.3846153846153846),
 (149, 157, 'http://en.wikipedia.org/wiki/Hey_Sandy', 0.3846153846153846)]

### C. _(25 points)_ Modifying html documents

__C1.__ _(10 points)_ Write a function called `wikify_html(path)` that does the following: 
1. Takes an local html file's path as input,
2. uses `BeautifulSoup` to loop over the `paragraph`s as strings, i.e., the string objects returned from the `.text` attributes of `<p>...</p>` tagged blocks, and
3. applies Wikifier to each,
4. writing the json annotation results, line by line (i.e., append mode) to a file with same path as the input file with the following suffix (extension) added: '.json'.

_Note_: If you have not completed __A2&mdash;A3__, you may complete this entire section for full credit using the packaged file: 
- `./data/example.htm`.

In [33]:
pathAlice = "./BookID19033.htm"

In [34]:
def wikify_html(path):
    with open(path, "r") as f: 
        html_file = f.read()
    soup = BeautifulSoup(html_file, 'html.parser')
    ptags = soup.find_all('p')
    annot = []
    for ptag in ptags:
        annot.append(Wikify(ptag.text))
    with open(path[:-4]+".json", 'w') as f:
        json.dump(annot, f)    

In [626]:
#do not run
wikify_html(pathAlice)

__C2.__ _(15 points)_ Finally (and this is a big one), we'd like to replace html paragraphs with ones that we've annotated with the URLs pointing to the Wikipedia articles. To do so, write a function called `get_enhanced_html(path)` that does the following:

1. Takes an html document's path as input,
2. checks to see if existing annotation data are already stored for the file (having the additional .json extension), 
    - running `wikify_html(path)` (from __C1__) if no annotations exist
3. loads the html document's `annotations` from file
4. retrieves the correct (by index) `annotation` from the `annotations` list,
5. uses `BeautifulSoup` to loop over the `paragraph`s, i.e., the `<p>...</p>` tagged blocks with the `enumerate()` function (to capture indices),
6. passes the `annotation` object through the `embed_links()` function, naming the output `annotated_paragraph_text`,
7. encloses `annotated_paragraph_text` in `<p>` tags and then applies `BeautifulSoup()`, storing the interpreted object as the `enhanced_paragraph`,
8. replaces each original `paragraph` with its corresponding `enhanced_paragraph` using the `.replace_with()` method for `BeautifulSoup` objects, and 
9. after the loop finishes, returns the enhanced html document (`BeautifulSoup` object).

Additionally, when done exhibit your function's output from application to the file under the path: `./data/example.htm`.

\[__Hint__: For Step 2, use the `os` module's function: `os.path.exists(path)`.\] 

__Note__: For more information on the `.replace_with()` method, check out the documentation at:
- https://www.crummy.com/software/BeautifulSoup/bs4/doc/#replace-with

In [81]:
def get_enhanced_html(path):
    if os.path.exists(path[:-4]+".json"):
        with open(path[:-4]+".json") as fh:
            Wikis = json.load(fh) #step3
    else:
        wikify_html(path)
        with open(path[:-4]+".json") as fh:
            Wikis = json.load(fh) #step3
############################################# 
    with open(path, "r") as f: 
        html_file = f.read()
    originalsoup = BeautifulSoup(html_file, 'html.parser')
    ptags = originalsoup.find_all('p')
    ptags_index_tuples = [(ptag, i) for i, ptag in enumerate(ptags)] #step 5
############################################# 
    annotations_and_num = [(wiki, len(wiki['annotations'])) for i, wiki in enumerate(Wikis)]
############################################# 
    annotated_paragraph_text = []
    for eachtuple in annotations_and_num:
        if eachtuple[1]:
            annotated_paragraph_text.append(embed_links(eachtuple[0]).data)
        else:
            annotated_paragraph_text.append(HTML(build_doc(eachtuple[0])).data)
############################################# 
    enhanced_paragraph = []
    for eachpar in annotated_paragraph_text:
        htmlpar = "<p>"+eachpar+"</p>"
        enhanced_paragraph.append(BeautifulSoup(htmlpar, 'html.parser')) #end of step 7
#############################################
    i = 0
    for p in originalsoup.find_all('p'):
        newparagraph = enhanced_paragraph[i]
        if p.string :
            p.string.replace_with(newparagraph)
        else:
            continue
        i += 1
        if i == len(ptags)-1:
            break
        else:
            continue
    with open(path[:-4]+"Enhanced.html", 'wb') as f:
        f.write(originalsoup.renderContents())
#############################################
    with open(path[:-4]+"Enhanced.html", "r") as f: 
        enhanced_html_file = f.read()

In [88]:
get_enhanced_html(pathAlice)
IFrame(pathAlice[:-4]+"Enhanced.html", width=800, height=200)

### D. _(25 points)_ Packaging and distribution

__D1.__  _(5 points)_ From a efficient access and distribution perspective, explain why __C1__'s Step 4 and __C2__'s Step 2 make this tool's processing minimal, and most of all, nice to the servers responding to our requests. Record this discussion in the response box below.

<font color=blue>Since a book has lots of paragraghs and it takes a while to wikify each paragragh seperately, if we have a way to run this part only once it would save a lot of computational power. Step 4 of C1 saves the results for future use and step 2 of C1 checks if the wikifier was already ran on the book, it will not produce the results again and only loads the file that has been saved.</font>

__D2.__ _(10 points)_ For distribution, it's important to have all your code packaged up nice and neat. So, take everything you've done in the previous steps and collect it all and place it so that it can be run with a single function call. Specifically create a function named `enhanced_book(book_id)` that does the following:
1. Inputs a `book_id`,
2. checks to see the book's file path exists locally, 
    - downloading an html copy using __A2__'s `download_book(book_id)` and storing locally if no local copy exists,
3. runs __C2__'s `get_enhanced_html(path)` function,
4. finally returning Step 3's `enhanced_html` output, exhibiting function by coercing the output to a string and applying the `HTML()` function.

__Important__: Completing the application of the full pipeline to an entire book from Project Gutenberg takes a long time (about 20-30 minutes for _Alice in Wonderland_). Make sure you test your code for function carefully, only when you're fairly sure it's completely done. Full credit will be awarded to attempts that have not been completely implemented, so if you run out of time for full testing please submit code anyway.

In [96]:
def enhanced_book(book_id):
    path = "./BookID"+book_id+".htm"
    if os.path.exists(path):
        pass
    else:
        save_html_file(book_id)
    get_enhanced_html(path)
    return IFrame(path[:-4]+"Enhanced.html", width=800, height=200)

In [97]:
enhanced_book("19033")

__D3.__ _(5 points)_ Now that you have a functioning and self-contained data pipeline, think of possible models for distribution of said data. Describe the data distribution model you believe to be the best. Should the software download and store all data locally on a user's machine, or should it always access data on the fly from the web?

<font color=blue>Downloading the whole data is not a very promising approach since it uses a lot of memory. Instead it is best to make a request for each data seperatly when they are needed. We can define a parent function which inputs the ID of the book and downloads the html file and calls all the related functions in order to produce the enhanced ebook. This function will save the nessesary files in between to avoid duplicate calculations. </font>

__D4.__ _(5 points)_ Finally, compare the possible advantages and weaknesses of each of the data distribution models you considered in the previous part.

<font color=blue>This approach makes a user friendly setup for creating the enhanced ebook and does not use a lot of memory on the user's machine. But on the other hand, it takes time to run the code for a whole book.</font>